# 4.2 Assistants


The Assistants API makes it possible to create AI assistants in applications. An Assistant has instructions and can use models, tools and files to answer the user's questions. The Assistants API currently supports three types of tools: Code Interpreter, File Search and Function Calling.


#

# 4.2.1 Assistant

In this example, an Assistant is created with the purpose of being a personal maths tutor.

Initially, an Assistant is created, which represents an entity that can be configured to respond to a user's messages using various parameters such as a , `name`, `instructions` and `model`.

In [1]:

from openai import OpenAI
from runners.standard_run import std_run
from runners.streaming_run import streaming_run


client = OpenAI()

assistant = client.beta.assistants.create(
    name="Math Tutor",
    instructions="You are a personal math tutor. Write and run code to answer math questions.",
    model="gpt-3.5-turbo",
)

Next, a Thread is created that represents a conversation between a user and one or more Assistants. A Thread can be created when a user (or their AI application) starts a conversation with their Assistant.

In [2]:
thread = client.beta.threads.create()

A message is added to the Thread, the content of the messages that users or applications create is added as objects to the Message Thread. 

In [3]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I need to solve the equation `8x + 12 = 14`. Can you help me?"
)

When the user has added all of the necessary information/messages to the thread you must run the thread to provide them with results.

You can run a thread using its id as well as the assistant's id, like this:

In [4]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    #model = "gpt-3.5-turbo" # By default the model defined in the assistant will be used. You can, however, overwrite it here.
    instructions = "Please address the user as SISTCA student." # This parameter is optional
)

During its lifecycle, a run can have multiple states:

![image.png](https://cdn.openai.com/API/docs/images/diagram-run-statuses-v2.png)
[https://platform.openai.com/docs/assistants/how-it-works/runs-and-run-steps]

Therefore, we must check for its completion.

In [9]:
if run.status == 'completed': 
    messages = client.beta.threads.messages.list(
    thread_id=thread.id
    )
    print(messages)
else:
    print(run.status)
    

    

queued


In [8]:
messages = client.beta.threads.messages.list(thread_id = thread.id)
print(messages.data[0].content[0].text.value)

SISTCA student, let's solve the equation `8x + 12 = 14` step by step.

1. Subtract 12 from both sides:
`8x + 12 - 12 = 14 - 12`
`8x = 2`

2. Divide by 8 to solve for x:
`8x / 8 = 2 / 8`
`x = 1/4`

Therefore, the solution to the equation `8x + 12 = 14` is `x = 1/4`.


In order to yield any results, an assistant's thread must be run. This is to say that whenever you're working with the assistant functionality you must have a runner function. 

Due to the fact that we will be exploring other types of assistants, and as a way of not repeating this code multiple times, we will export as the following function:

In [ ]:
def std_run (thread_id, assistant_id, client):
    run = client.beta.threads.runs.create_and_poll(
        thread_id = thread_id,
        assistant_id = assistant_id,
        instructions = "Please address the user as SISTCA student."
    )   

    if run.status == 'completed': 
        messages = client.beta.threads.messages.list(thread_id = thread_id)
        print(messages.data[0].content[0].text.value)
    else:
        print(run.status)

If you have used ChatGPT before you may have noticed that the output is not returned all at once, like it is here, but rather gradually, in real time.

We can achieve this same effect by using a different type of runner, the streaming run.


In this run, the streaming_run function is defined, which is responsible for starting the streaming with the assistant. 
Within this, the 'EventHandler' class is defined, which is inherited from AssistantEventHandler. 
This class contains the methods responsible for dealing with the different events that occur during streaming: 

* `on_text_created`: Called when a new text is created by the assistant. This prints 'assistant' to indicate that the assistant is responding.

* `on_text_delta`: Called for each text update. This prints the part of the text that was generated (delta.value).

* `on_tool_call_created`: Called when the wizard calls a tool. This prints the type of tool call (tool_call.type).

* `on_tool_call_delta`:  Called for tool call updates, specifically for the code_interpreter. This prints the code_interpreter input and any outputs, especially logs.


Finally, the API client's create_and_stream method is used to start streaming, passing the EventHandler as the event handler.


* `thread_id`: and`assistant_id` are the required identifiers.
* `instructions` defines specific instructions for the assistant, in this case asking it to address the user as a SISTCA student.
* `event_handler` is the instance of the EventHandler class.

The `with` block ensures that streaming continues until it is finished (stream.until_done()).

In [7]:
# With streaming

from typing_extensions import override
from openai import AssistantEventHandler


def streaming_run (thread_id, assistant_id, client):

    # First, we create a EventHandler class to define
    # how we want to handle the events in the response stream.
    
    class EventHandler(AssistantEventHandler):    
        @override
        def on_text_created(self, text) -> None:
            print(f"\nassistant > ", end="", flush=True)
            
        @override
        def on_text_delta(self, delta, snapshot):
            print(delta.value, end="", flush=True)
            
        def on_tool_call_created(self, tool_call):
            print(f"\nassistant > {tool_call.type}\n", flush=True)
        
        def on_tool_call_delta(self, delta, snapshot):
            if delta.type == 'code_interpreter':
                if delta.code_interpreter.input:
                    print(delta.code_interpreter.input, end="", flush=True)
            if delta.code_interpreter.outputs:
                print(f"\n\noutput >", flush=True)
                for output in delta.code_interpreter.outputs:
                    if output.type == "logs":
                        print(f"\n{output.logs}", flush=True)


    # Then, we use the `create_and_stream` SDK helper 
    # with the `EventHandler` class to create the Run 
    # and stream the response.
    
    with client.beta.threads.runs.stream(
        thread_id=thread_id,
        assistant_id=assistant_id,
        instructions="Please address the user as SISTCA student.",
        event_handler=EventHandler(),
    ) as stream:
        stream.until_done()

In [10]:
print(streaming_run(thread.id, assistant.id, client))


assistant > SISTCA student, let's solve the equation `8x + 12 = 14` step by step.

1. Subtract 12 from both sides:
`8x = 14 - 12`
`8x = 2`

2. Divide by 8 to solve for x:
`x = 2/8`
`x = 1/4`

Therefore, the solution to the equation `8x + 12 = 14` is `x = 1/4`.None


# 4.2.2 Function Calling

Function Calling is one of Assistants API's tools, which in simple terms, invokes predefined functions in order to respond to a certain prompt. As you will verify in just a moment, in this tutorial we asked our weather assistant to give us the maximum and minimum temperatures and rain probability in a certain location.  

By using functions we allow the assistant to call methods the add extra functionality as it sees fit. 

For instance, in this case we want to make it so that when asking for weather data for a Portuguese city or region the data should come from [IPMA](https://www.ipma.pt/pt/index.html).

By using [IPMA'S API](https://api.ipma.pt/) we can check the weather data for a specific location using it's ID.



We start by defining the functions the assistant may call, alongside their parameters.

In [ ]:
assistant = client.beta.assistants.create(
    instructions="You are a weather bot. Use the provided functions to answer questions.",
    model="gpt-3.5-turbo",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "get_weather_data",
                "description": "Get the current weather forecast for the specified location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "City or region for which to get the weather forecast",
                        }
                    },
                    "required": ["location"]
                }
            }
        }
    ]
)


The following function, `get_location_id`, takes `location_name` as input and returns an id of the specific location. First of all, we create an `url`, which will be accessed in order to retrieve the actual data. To actually receive this data, we use a GET HTTP request, the `response` will then be converted to a json variable, in this case, `data`. As we now have access to all data retrieved from the GET request, now all we need is to just return the `location_id`.  

In [ ]:
import requests
def get_location_id(location_name):
    url = "https://api.ipma.pt/open-data/distrits-islands.json"
    try:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            for location in data["data"]:
                if location["local"].lower() == location_name.lower():
                    return str(location["globalIdLocal"])
            return None  # Location not found
        else:
            print("Failed to fetch data:", response.status_code)
            return None
    except Exception as e:
        print("An error occurred:", e)
        return None

Our next function, `get_weather_data()`, is very similar to `get_location_id()` as we simply use the returned variable, `location_id` to access the actual weather data of the required location, which it returns as `forecast`.

In [ ]:
def get_weather_data(location_id):
    url = f"https://api.ipma.pt/open-data/forecast/meteorology/cities/daily/{location_id}.json"
    try:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            forecast = data["data"][0]
            return forecast
        else:
            print("Failed to fetch data:", response.status_code)
            return None
    except Exception as e:
        print("An error occurred:", e)
        return None